# Домашнее задание №1: линейная регрессия (максимум 10 баллов)

Некоторые задания будут по вариантам (всего 4 варианта). Чтобы выяснить свой вариант, посчитайте количество букв в своей фамилии, возьмете остаток от деления на 4 и прибавьте 1.

In [35]:
import numpy as np
print('My variant is', len('Nurullin')%4+1)

My variant is 1


## Многомерная линейная регрессия из sklearn

Создадим набор данных для многомерной регрессии

In [4]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples = 10000)
print(X.shape, y.shape)

(10000, 100) (10000,)


У нас 10000 объектов и 100 признаков. Для начала решим задачу аналитически "из коробки".

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X, y)
print(mean_squared_error(y, reg.predict(X)))

2.3816291655653714e-25


Теперь попробуем обучить линейную регрессию методом градиентного спуска "из коробки"

In [6]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(alpha=0.00000001).fit(X, y)
print(mean_squared_error(y, reg.predict(X)))
reg.coef_

3.3043486091961774e-12


array([ 1.34729853e-08, -4.29843662e-08,  3.98497322e-08,  5.89662903e-08,
        2.90767650e+01,  3.18894632e-08, -3.99833392e-08, -2.69441716e-09,
        6.94140069e+01, -2.64269448e-08,  1.34698853e-08, -8.61501732e-09,
       -2.81073594e-09,  7.64336917e+01, -3.34615737e-08,  6.11485473e-09,
        3.04716044e-08,  7.99649209e-08,  1.93735605e-09,  2.85389122e-08,
       -3.38181107e-08, -4.80362379e-09, -2.16407343e-08,  1.40467593e-08,
        4.96399051e-08, -2.35903552e-08,  2.62286479e-08, -7.08143490e-09,
        2.60277106e-08,  2.11942325e-08,  6.45710686e-08, -3.81760643e-08,
       -4.62978761e-08, -1.14285137e-08, -1.04349721e-08, -4.29251684e-08,
       -8.28129539e-08, -5.76639177e-08,  8.74741503e-09,  1.39972416e-08,
       -6.04385519e-09, -5.14225428e-09, -3.52367299e-08,  6.34308899e+01,
       -3.05487861e-09, -1.14381906e-08, -8.12381902e-09, -3.45311210e-08,
       -3.11557823e-09,  1.82319081e-08,  7.58480855e-09,  1.81336847e-09,
        1.19081085e-09, -

***Задание 1 (1 балл).*** Объясните, чем вызвана разница в значениях двух полученных значений метрики?

***Задание 2 (1 балл).*** Подберите гиперпараметры в методе градиентного спуска так, чтобы значение MSE было близко к значению MSE, полученному при обучении LinearRegression.

1) LinearRegression() находит решение линейной регрессии аналитически, а стохастический градиент - "методом тыка", поэтому логично, что ошибка аналитической линейной регрессии будет на несколько порядков ниже ошибки стохастического метода.


2) Есть два способа улучшения качества модели. Первое - понижение альфы. Чем меньше alpha, тем меньше штраф (регуляризация), что позволяет модели лучше адаптироваться к данным (тут главное не переобучить). Второе - увеличение количества доступных итераций в функци SGDRegressor

Однако стоит отметить, что очень маленькая alpha может привести к медленному обучению. Баланс между alpha и количеством итераций — ключ к хорошей модели


In [7]:
#Пусть альфа равна 1e-20 - на 9 порядков меньше альфы из примера выше

reg = SGDRegressor(alpha=1e-20, max_iter=100000000).fit(X, y)

mean_squared_error(y, reg.predict(X))

7.607270071553692e-25

## Ваша многомерная линейная регрессия

***Задание 3 (5 баллов)***. Напишите собственную многомерную линейную регрессию, оптимизирующую MSE методом *градиентного спуска*. Для этого используйте шаблонный класс.

Критерий останова: либо норма разности весов на текущей и предыдущей итерациях меньше определенного значения (первый и третий варианты), либо модуль разности функционалов качества (MSE) на текущей и предыдущей итерациях меньше определенного значения (второй и четвертый варианты). Также предлагается завершать обучение в любом случае, если было произведено слишком много итераций.

***Задание 4 (2 балла)***. Добавьте l1 (первый и четвертый варианты) или l2 (второй и третий варианты) регуляризацию.

In [111]:
class LinearRegression(object):
    def __init__(self, alpha=0.0001, l_ratio=0.001, tol=0.001, max_iter=1000):
        self.alpha = alpha
        self.l_ratio = l_ratio
        self.tol = tol
        self.max_iter = max_iter

    def fit(self, X, y):
        n_samples, n_features = X.shape   #(10000, 100)
        
        self.weights = np.zeros(n_features) #задаем начальные веса равными 0
        self.bias = 0                       #Начальное смещение = 0
        
        prev_weights = np.zeros(n_features) #Предыдущие веса для отслеживания изменений
        prev_mse = 1e+20              # Предыдущее значение MSE - на шару что нибудь большое влепил
        
        for iteration in range(self.max_iter):   #1000 итераций
            y_pred = np.dot(X, self.weights) + self.bias #получаем вектор предсказаний длиной n_samples
               
            errors = y_pred - y #Расчет ошибок
            #расчет grad - векторов
            grad_weight = np.dot(X.T, errors) / n_samples
            grad_bias = np.sum(errors) / n_samples
            
            #L1 регуляризация
            grad_weight += self.l_ratio * np.sign(self.weights)
            
            self.weights -= self.alpha * grad_weight
            self.bias -= self.alpha * grad_bias
            
            mse = np.sum(errors**2) / n_samples
            
            #Критерий остановки
            weight_diff = np.linalg.norm(self.weights - prev_weights)
            
            if weight_diff < self.tol:
                print('ура ура сошлось')
                break
            
            # Обновление предыдущих значений
            prev_weights = np.copy(self.weights)
            prev_mse = mse

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias


    

In [112]:
my_reg = LinearRegression()
my_reg.fit(X, y)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y, my_reg.predict(X))
print(f'MSE: {mse}')

MSE: 29487.950583963997


Получается какая то беда, условие останова даже не выполняется, попробуем взять скорость обучения побольше (также можно сделать больше итераций)

In [115]:
my_reg = LinearRegression(alpha=0.1)
my_reg.fit(X, y)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y, my_reg.predict(X))
print(f'MSE: {mse}')

assert mean_squared_error(y, my_reg.predict(X)) < 1e-3
print('You are amazing! Great work!') 
#more like it

ура ура сошлось
MSE: 1.45130184084742e-05
You are amazing! Great work!


***Задание 5 (1 балл)***. Обучите линейную регрессию из коробки с l1-регуляризацией (from sklearn.linear_model import Lasso, первый и четвертый варианты) или с l2-регуляризацией (from sklearn.linear_model import Ridge, второй и третий варианты) с значением параметра регуляризации 0.1. Обучите вашу линейную регрессию с тем же значением параметра регуляризации и сравните результаты. Сделайте выводы.

In [81]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso

lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(X, y)

y_pred_lasso = lasso_reg.predict(X)
mse_lasso = mean_squared_error(y, y_pred_lasso)
print(f'MSE for Lasso (sklearn): {mse_lasso}')

my_lasso_reg = LinearRegression()
my_lasso_reg.fit(X, y)

y_pred_my_lasso = my_lasso_reg.predict(X)
mse_my_lasso = mean_squared_error(y, y_pred_my_lasso)
print(f'MSE for Lasso (DIY variant): {mse_my_lasso}')


MSE for Lasso (sklearn): 0.10082940910578796
MSE for Lasso (DIY variant): 29487.950583963997


In [84]:
my_lasso_reg = LinearRegression(alpha=0.01)
my_lasso_reg.fit(X, y)

y_pred_my_lasso = my_lasso_reg.predict(X)
mse_my_lasso = mean_squared_error(y, y_pred_my_lasso)
print(f'MSE for Lasso (sklearn): {mse_lasso}')
print(f'MSE for Lasso (DIY variant): {mse_my_lasso}')

ура ура сошлось
MSE for Lasso (sklearn): 0.10082940910578796
MSE for Lasso (DIY variant): 0.010705266376535039


в общем, выбранное значение learing rate как будто бы не очень хорошо предсказывает, поэтому я взял alpha=0.01, и получил результаты луше 

судя по результатам, сам sklearn использует значение альфа примерно равное 0.01

В то же время, если взять alpha=0.1 (см. пункт выше), результат мсе получится еще меньше -> выше качество.


